In [1]:
import numpy as np
import pandas as pd
import scipy
import sklearn.metrics
import os

import matplotlib.pyplot as plt
import matplotlib as mpl
#import seaborn as sns

from sklearn.metrics import roc_auc_score
import statsmodels.stats.multicomp as mc

# Collect predictions

In [2]:
regressor = 'MLP'
merge_embeddings = 1

In [3]:
data_dir = f'/lustre/groups/epigenereg01/workspace/projects/vale/mlm/variants/predictions/merge_embeddings_{merge_embeddings}/{regressor}/'

In [4]:
#models = ['Species-aware','Species-agnostic','DNABERT','DNABERT-2','NT-MS-v2-500M','4mers','5mers','word2vec','effective_length']
models = ['dnabert','dnabert-3utr',  'dnabert2', 'dnabert2-3utr',  'ntrans-v2-250m',  'ntrans-v2-250m-3utr', 'stspace', 'stspace-spaw',
          '5mers', 'effective_length', ]

In [5]:
res = {}

for subset in ('clinvar', 'gnomAD','eQTL-susie','eQTL-GRASP'):
        res[subset] = []
        for model in models:
            res_tsv = data_dir + f'{subset}-{model}.tsv'
            if os.path.isfile(res_tsv):
                df = pd.read_csv(res_tsv, sep='\t')
                df['model'] = model
                res[subset].append(df)
        if len(res[subset])>0:
            res[subset] = pd.concat(res[subset])
            N=res[subset].groupby('model').size().mean()
            print(subset)

clinvar
gnomAD
eQTL-susie
eQTL-GRASP


In [6]:
preds_res = {}
best_models = {}


for subset in ('clinvar', 'gnomAD','eQTL-susie','eQTL-GRASP'):

        if len(res[subset])>0:
        
            preds_res[subset] = res[subset].groupby('model').apply(lambda x:roc_auc_score(x.label.values,x.y_pred.values))
        
preds_res = pd.DataFrame(preds_res).map(lambda x: f'{x[0]:.2f}±{x[1]:.2f}' if type(x)==tuple else x)

preds_res

,clinvar,gnomAD,eQTL-susie,eQTL-GRASP
model,,,,
dnabert,0.937998,0.595723,0.571797,0.627077
dnabert-3utr,0.943638,0.606252,0.580178,0.647999
dnabert2,0.957653,0.597439,0.686390,0.659115
dnabert2-3utr,0.932629,0.549542,0.629469,0.608714
ntrans-v2-250m,0.916932,0.563236,0.593616,0.591837
ntrans-v2-250m-3utr,0.886992,0.560595,0.594009,0.590018
stspace,0.941299,0.586070,0.627471,0.639632
stspace-spaw,0.940304,0.584976,0.620793,0.625126
